## feature extraction exercise
```This exercise is purely about features extraction. We will learn how to do it quick and efficiently.
We will be working on a kaggle dataset of quora questions, where each record is composed of a pair of questions, while the target is to determine whether the questions have the same meaning.
We will extract features for each question and for each pair of questions and will train a simple model (default xgboost) using those features.```

```The purpose of this exercise is to acquire good practices, so please read the instructions carefully and do as it says. You are also encouraged to look at the solution when after you are finished. In addition, when solving the exercise, try to write
as efficient and as clean code as you can.```

```Note: We are about to do some kaggle cheats, that is, we will compute features by mixing the train and the test.
Please notice exactly where we did so and tell it to your tutor. In addition, every time you meet a question in the instructions (you can identify a question by '?'), please answer it in a comment block for future discussing with you tutor. ```

```~Ittai Haran```

In [ ]:
# some modules you might find useful

import pandas as pd
import numpy as np
from collections import Counter
from functools import partial
import re
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# sentence tokenizer for future use

from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
twt_tokenizer = TweetTokenizer()

# word2vec model for future use

from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('resources/word_2_vec/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
data = pd.read_csv('resources/data/train.csv')
data.head()
data = data.iloc[:2000]
data = data[data.apply(lambda x: not (type(x['question1']) == float or type(x['question2']) == float), axis = 1)]

```First we would like to extract features regarding a single question. In order to do so, first create a dataset containing  all the questions (and their id. why should we remember the id?), without duplicates. Name it 'questions'.```

``` Add a column containing the questions, tokenized using twt_tokenizer, the TweetTokenizer object we created earlier. Name it 'question_sep'. Make sure that you treat the questions in lower case.```

```Create an empty list called 'question_features_for_future_use'. We are going to befoul the questions dataframe, so we will want to remember which of its columns are important to us and which are just columns helping us to create other columns.```
```Next, I will ask you to create some features. Whenever I use this sign: (*), know that you have to add the feature name to the list.```

```Before we start computing features, write a function that gets a column name and saves a csv file with 2 columns: qid and the column chosen. Name it 'save_feature' and make sure you use it after every feature computed, since it might be very very important for later parts of the exercise and your life.```

```Save the features in the resources/features/<col_name>.csv .```

```use os.path and os.getcwd().```

```Compute the following:```
- ```Counter of the word part of speech (use collections.Counter, which we imported earlier. do it using one line). (*)```
- ```number of different numbers appearing in the question
(numbers, not digits. use regex. don't count words like 'one') (one line). (*)```
- ```number of words in a question (one line). (*)```
- ```length of longest word (one line). (*)```
- ```word2vec mean of the question. (*)```

- ```Counter of the question_words (one line). (*)```

In [ ]:
question_words = ['why', 'how', 'where', 'who', 'what', 'which', 'when', 'wheather']

```We will now use tf-idf grade (if you aren't familiar with the concept, read about it, and be ready to
talk about it with your tutor ;) ``` https://en.wikipedia.org/wiki/Tf%E2%80%93idf ```).
do the following:```
- ```initialize a TfidfVectorizer object. use norm = None, use English stop words and twt_tokenizer we used before. Name it tfidf.```
- ```create the tf-idf matrix of all the questions (look again at the note in the beginning of the exercise).```
- ```look at tfidf.vocabulary_.```
- ```create a reversed vocabulary (given an index returns a word. do it in one line, using list comprehension).```
- ```create a column, such that every question has a list of its words and the word's tf-idf grades. do it without transferring the tf-idf matrix into a dense matrix (keep it sparse).```
- ```for each question, find the third biggest tf-idf grade. create a column such that every question have a list of the words with bigger grades the the third biggest tf-idf grade. (*)```

```We will do it again, this time more general, by completing the following functions.```

In [ ]:
def get_most_important_words_n(x, n = 3):
    # the function gets a list of tuples (word, tf-idf grade) and n
    # it returns the words which have grades bigger than the n'th biggest grade. Note that several words may have the same grade.
    pass

def get_most_important_words_total(x, n = 3):
    # the function gets a list of tuples (word, tf-idf grade) and n
    # it returns the n words that have the biggest grades
    pass

In [ ]:
def get_tfidf_features(n_gram, most_important_words_method):
    # the function gets the size of n for the n_grams in the tf-idf computation and
    # the result method, which will be one of the two functions above
    # it adds the tf-idf features you computed before
    # when n_gram = 1, make the function add another feature: the mean word2vec vector of the most important words
    # features created will be added to question_features_for_future_use (!)
    pass

In [ ]:
get_tfidf_features(1,get_most_important_words_n)
get_tfidf_features(2,partial(get_most_important_words_total, n=3))

```We now move to features concerning both questions, and not just one of them. But first, run the following cell, known as the evil cell.```

In [ ]:
exec(''.join(map(lambda x: chr(ord(x)-1), 'fyju)*'))+'\n'\
     +''.join(map(lambda x: chr(ord(x)-1), 'qsjou)#Ibibibib!J!fyjufe!zpvs!lfsofm/!Dpoujovf!gspn!ifsf'+\
                  '!xjuipvu!sfsvoojoh!uif!qsfwjpvt!dfmmt!)cftjeft!uif!jnqpsu!dfmmt-!boe!mpbe!uif!ebub!bhbjo*#*'))\
    +'\n'+''.join(map(lambda x: chr(ord(x)-1), 'jnqpsu!nbuqmpumjc/qzqmpu!bt!qmu\x0bgspn!nbuqmpumjc/jnbhf!jnqpsu!jnsfbe\x0bjnb'+\
                      'hf!>!jnsfbe)#sftpvsdft0wjtvbmj{bujpo!ifmqfst0T'+\
                      'njmjoh`Efwjm`Fnpkj/qoh#*\x0bqmu/jntipx)jnbhf*\x0bqmu/tipx)*')))

```Understand how the evil cell works.```

```Now we will add the features we computed earlier to the data dataframe. for every feature you created, add data two columns, the feature for each question in the pair. Use DataFrame.merge and the qid columns you saved every time you saved a feature. Use also os.listdir and DataFrame.rename, and do it in 7 lines of code at top.
Use the following converter (in the pd.read_csv syntax): converters = {feature_name:lambda x: eval(x)}. Why is it needed? Hint: open pos_tag_count.csv. If you aren't familiar with the amazing eval function, read about it and be ready to discuss it with your tutor :)```

``` Now we would like to find a way to take a feature for each question separately and make it one. Remember our question features are of 3 kinds:```
- ```number```
- ```Counter```
- ```vector```

```For each kind we will write a method taking both features and producing one feature:```

In [ ]:
def from_two_features_to_1_number(number_1, number_2):
    # the function returns |number_1-number_2|
    pass

def from_two_features_to_1_vector(vector_1, vector_2):
    # the function returns the cosine similarity between the vectors
    pass

def from_two_features_to_1_counter(counter_1, counter_2):
    # the Counter objects represent distribution. return the sim of all absolute values of differences between them.
    # Remember- Counter have some great properties.
    pass

def from_two_features_to_1(feature_1, feature_2):
    # return the right one feature based on the feature's types
    pass

```I suspect you know what that's for:```

In [ ]:
data_features_for_future_use = []

```Use the methods you wrote to get one feature from every pair of features you have, while running over the features in question_features_for_future_use. give it meaningful names.```

```Add the following features:```
- ```number of common words between the two questions. (one line) (*)```
- ```number of common words between the two questions, not including stop words. (one line) (*)```

```You might have to use twt_tokenizer again. note that we could save the tikenized questions.```

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

```Now think of a feature of your own and implement it.```

```I'm not going to use the evil cell again, but I'll remind you to save your features.```

```That's it! take your features and train a RandomForestRegressor using them. Don't forget to split to train and test sections. What score did you get?```

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier